<a href="https://colab.research.google.com/github/InnocentNewjeans/InterviewAnswerMaker/blob/Model/Model/llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "NousResearch/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    torch_dtype='auto',
    device_map='auto',
    do_sample=True,
    load_in_4bit=True
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def correct_question(text):
    chat = [
        {"role": "system", "content": "You are an AI assistant that corrects interview questions in Korean by making the subject clear and fixing any grammatical errors. Provide only the corrected question without any notes or explanations."},
        {"role": "user", "content": f"{text}"},
        {"role": "assistant", "content": ""}
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            max_new_tokens=256,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
    return output.strip()


In [ ]:
text = "스터디를 따로 하고 있는 것이 있나요 있다면 그 공부를 왜 하고 있는지 그 이유도 같이 말씀해 주세요"
corrected_question = correct_question(text)
print(corrected_question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


면접관: 스터디를 따로 하고 있는 것이 있나요? 있다면 그 공부를 왜 하고 있는지 그 이유도 같이 말씀해 주세요?assistant

면접관: 스터디를 따로 하거나 하고 있는 것이 있나요? 있다면 그 공부를 왜 하고 있는지 그 이유도 같이 말씀해 주세요?assistant

면접관: 스터디를 따로 하거나 하고 있는 것이 있나요? 있다면 그 이유도 같이 말씀해 주세요?assistant

면접관: 스터디를 따로 하거나 하고 있는 것이 있나요? 있다면 그 이유도 함께 말씀해 주세요?assistant

면접관: 따로 스터디를 하거나 하고 있는 것이 있나요? 있다면 그 이유를 함께 설명해 주세요?assistant

면접관: 따로 스터디를 하거나 하는 것이 있나요? 있다면 그 이유를 함께 설명해 주세요?assistant

면접관: 따로 스터디를 하는 것이 있나요? 있다면 그 이유를 함께 설명해 주세요?assistant

면접관: 따로 스터디를
